# ResNet18

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
from torchvision.models import densenet121 
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
transform = transforms.Compose([
    transforms.Grayscale(1),  
    transforms.Resize((48, 48)),  
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),  
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1), 
    transforms.ToTensor() 
])

train_dataset = ImageFolder(root='/kaggle/input/fer2013/train', transform=transform)
test_dataset = ImageFolder(root='/kaggle/input/fer2013/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, dropout_rate=0.5):  
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.downsample(identity)
        out = self.relu(out)
        return out
    
class DeeperResNet(nn.Module):
    def __init__(self, block, layers, num_classes=7, dropout_rate=0.5): 
        super(DeeperResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, num_classes)


    def _make_layer(self, block, out_channels, blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeeperResNet(BasicBlock, [2, 2], num_classes=7, dropout_rate=0.3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
train_losses = []
test_losses = []
test_accuracies = []
train_accuracies = []

num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for i, (inputs, labels) in enumerate(train_loader, 0):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f - Train Accuracy: %.2f %%' %
                  (epoch + 1, i + 1, running_loss / 100, (100 * correct_train / total_train)))
            running_loss = 0.0

    scheduler.step()

    model.eval()
    correct_test = 0
    total_test = 0
    test_running_loss = 0.0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

            test_loss = criterion(outputs, labels)
            test_running_loss += test_loss.item()

        avg_test_loss = test_running_loss / len(test_loader)
        test_losses.append(avg_test_loss)

        test_accuracy = (100 * correct_test) / total_test
        test_accuracies.append(test_accuracy)

    train_accuracy = (100 * correct_train) / total_train
    train_accuracies.append(train_accuracy)
    train_losses.append(running_loss)


highest_test_accuracy = max(test_accuracies)
least_test_loss = min(test_losses)

print(' Accuracy on the test images: %.2f %%' % highest_test_accuracy)
print(' Loss on the test images: %.4f' % least_test_loss)


In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Test Losses')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Test Accuracies')
plt.legend()

plt.tight_layout() 
plt.show()


In [ ]:
torch.save(model.state_dict(), 'ResNet18Adam.pth')

## SGD

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeeperResNet(BasicBlock, [2, 2], num_classes=7, dropout_rate=0.3).to(device)  
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-5)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# Lists to store training loss, test loss, test accuracy, and train accuracy
train_losses = []
test_losses = []
test_accuracies = []
train_accuracies = []

num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for i, (inputs, labels) in enumerate(train_loader, 0):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f - Train Accuracy: %.2f %%' %
                  (epoch + 1, i + 1, running_loss / 100, (100 * correct_train / total_train)))
            running_loss = 0.0

    scheduler.step()

    # Testing accuracy and loss after each epoch
    model.eval()
    correct_test = 0
    total_test = 0
    test_running_loss = 0.0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

            test_loss = criterion(outputs, labels)
            test_running_loss += test_loss.item()

        avg_test_loss = test_running_loss / len(test_loader)
        test_losses.append(avg_test_loss)

        test_accuracy = (100 * correct_test) / total_test
        test_accuracies.append(test_accuracy)

    train_accuracy = (100 * correct_train) / total_train
    train_accuracies.append(train_accuracy)
    train_losses.append(running_loss)


highest_test_accuracy = max(test_accuracies)
least_test_loss = min(test_losses)

print(' Accuracy on the test images: %.2f %%' % highest_test_accuracy)
print(' Loss on the test images: %.4f' % least_test_loss)


In [ ]:

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Test Losses')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Test Accuracies')
plt.legend()

plt.tight_layout()  
plt.show()

In [ ]:
torch.save(model.state_dict(), 'ResNetSGD.pth')